In [1]:
import pandas as pd
import numpy as np
import time
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen

In [2]:
url = 'https://www.instagram.com/explore/'

In [13]:
#설정값
path = 'C:/Users/ASUX/Desktop/chromedriver_win32/chromedriver.exe' #chromedriver 위치

ID = '' #인스타 id
PW = '' #인스타 비밀번호

SCROLL_MAX = 10000 #얼마나 스크롤할 지 정하기
CRAWLING_NUM = 5 #크롤링할 게시물 개수

In [4]:
def scroll():
    body = driver.find_element_by_tag_name('body')

    while True:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.01) #스크롤 속도 조절
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        scroll_len = soup.find(class_='K6yM_').find('div')["style"]
        scroll_len = float(re.search('padding-top: (.*)px;', scroll_len).group(1))
        
        if scroll_len > SCROLL_MAX:
            break

In [5]:
def select_first(driver):
    first = driver.find_element_by_css_selector('._9AhH0')
    first.click()
    time.sleep(1)

In [16]:
img_names = []
img_srcs = []
contents = []
places = []
tags = []

n = 1

data = {'img_name': img_names,
        'img_src': img_srcs,
        'content': contents,
       'place': places,
       'tag': tags,
       }

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    
    ###이미지
    url = driver.current_url
    driver.execute_script("window.open();") #새 탭에서 열기
    driver.switch_to.window(driver.window_handles[1])
    driver.get(url)
    time.sleep(2)
    
    html_temp = driver.page_source
    soup_temp = BeautifulSoup(html_temp, 'lxml')
    
    try:
        soup_temp.find("div", class_='fXIG0')["class"] #영상 제외
            
    except:
        soup_temp = soup_temp.find("div", class_='KL4Bh')
        img_src = soup_temp.find("img")["src"] #이미지가 두 개이상인 경우도 맨 처음 이미지만 수집됩니다.
        img_srcs.append(img_src)
        
        global n
        img_name = 'img_' + str(n) + '.jpg'
        img_names.append(img_name)
        
        with urlopen(img_src) as f:
            with open('./img/' + 'img_' + str(n) + '.jpg', 'wb') as h:
                img = f.read()
                h.write(img)
    
    else:
        img_srcs.append(np.nan)
        img_names.append(np.nan)
        
    n += 1
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    
    ###내용
    content = soup.select('div.C4VMK > span')[0].text
    if content == '':
        contents.append(np.nan)
    else:
        contents.append(content)
            
            
    ###해시태그
    tag = soup.select('a.xil3i') #댓글에 있는 해시태그도 다 수집되는 것 같아요.
    tags_temp = []
    for i in tag:
        tags_temp.append(i.get_text())
        
    if tags_temp == []:
        tags.append(np.nan)
    else:
        tags.append(''.join(tags_temp))
           
        
    ###위치
    place = soup.select('div.JF9hh')[0].text
    if place == '':
        places.append(np.nan)
    else:
        places.append(place)
            
    
    print("{}개 완료".format(n))
    return data

In [7]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()

In [14]:
driver = webdriver.Chrome(path)
driver.get(url)
time.sleep(1)

In [15]:
#login
login_id = driver.find_element_by_name('username')
login_id.send_keys(ID)
login_pw = driver.find_element_by_name('password')
login_pw.send_keys(PW)
login_pw.send_keys(Keys.RETURN)
time.sleep(3)

In [17]:
#먼저 스크롤 해두기
scroll()
time.sleep(3)

#맨 위로 올라가기
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(3)

#크롤링 시작
select_first(driver)

num = CRAWLING_NUM

for n in range(num):
    data = get_content(driver)
    time.sleep(1)
    move_next(driver)
    time.sleep(1)

1개 완료
2개 완료
3개 완료
4개 완료
5개 완료


In [18]:
df = pd.DataFrame.from_dict(data)

In [19]:
df

,img_name,img_src,content,place,tag
0,NaN,NaN,"-졸려도 치카포카 해야대여 겅즈님,,,#공주님의양치 #따쓰냄새방지위...",NaN,#공주님의양치#따쓰냄새방지위원회#양치만렙#mabell_...
1,img_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,#광고 세일러문 x ost 콜라보 반지갑 ✨투피...,NaN,#광고#상품협찬#세일러문#미소녀전사세이...
2,img_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,이름도 예쁜 #해리단길 데이트!갬성 카페부터 다양한 맛집까지 한번에♥완전 풀코스 데...,NaN,#해리단길#수수굉#투와이스#해리단길#해리단길맛집#해리단길카페#유루유루#볼라바치오#태...
3,img_3.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,⠀2021.02.15#요거트코울슬로 #그림의빵최애빠...,NaN,#요거트코울슬로#그림의빵#디디미니레시피도ᄌ...
4,img_4.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,업 데 이 트#룩플,NaN,#룩플
